In [2]:
!pip install pandas scikit-learn

/bin/bash: /home/vtpldedpy/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd

df = pd.read_csv("./dataset/train.csv")

In [4]:
df.head(3)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = df.drop('label', axis=1).values
y = df['label'].values

In [6]:
X.shape

(42000, 784)

In [7]:
y.shape

(42000,)

In [8]:
from sklearn.model_selection import train_test_split

# X_subset, _, y_subset, _ = train_test_split(X, y, train_size=20000, stratify=y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC())
])

param = {
    "svm__C":[0.1, 1, 10],
    "svm__gamma":['scale', 'auto'],
    "svm__kernel":["linear", "rbf", "poly"]
}

In [10]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipeline,param, cv = 5, scoring="accuracy", n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('svm', SVC())]),
             n_jobs=-1,
             param_grid={'svm__C': [0.1, 1, 10],
                         'svm__gamma': ['scale', 'auto'],
                         'svm__kernel': ['linear', 'rbf', 'poly']},
             scoring='accuracy')

In [11]:
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_:.4f}")

Best parameters: {'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'poly'}
Best cross-validation accuracy: 0.9697


In [12]:
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm', SVC(C=10, kernel='poly'))])

In [13]:
test_accuracy = best_model.score(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")

Test accuracy: 0.9718


In [14]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

In [21]:
knn_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier())
])

knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

In [22]:
knn_clf = KNeighborsClassifier()

In [23]:
grid_search_knn = GridSearchCV(estimator=knn_clf, param_grid=knn_param_grid, cv=5, n_jobs=-1, verbose=2)

In [24]:
grid_search_knn.fit(X_train, y_train)

Fitting 5 folds for each of 224 candidates, totalling 1120 fits


GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'metric': ['euclidean', 'manhattan', 'chebyshev',
                                    'minkowski'],
                         'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
                         'weights': ['uniform', 'distance']},
             verbose=2)

In [25]:
best_params_knn = grid_search_knn.best_params_
best_score_knn = grid_search_knn.best_score_

print(f"Best parameters found: {best_params_knn}")
print(f"Best cross-validation score: {best_score_knn}")

Best parameters found: {'algorithm': 'auto', 'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}
Best cross-validation score: 0.9654285714285713


In [26]:
best_model_knn = grid_search_knn.best_estimator_
best_model_knn.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=3, weights='distance')

In [27]:
test_accuracy_knn = best_model_knn.score(X_test, y_test)
print(f"Test accuracy: {test_accuracy_knn:.4f}")

Test accuracy: 0.9662


In [28]:
# dt
from sklearn.tree import DecisionTreeClassifier

In [29]:
dt_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("dt", DecisionTreeClassifier())
])

dt_param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 5, 10],
    'max_features': [None, 'auto', 'sqrt', 'log2']
}

In [30]:
dt_clf = DecisionTreeClassifier()

In [31]:
grid_search_dt = GridSearchCV(estimator=dt_clf, param_grid=dt_param_grid, cv=5, n_jobs=-1, verbose=2)

In [33]:
grid_search_dt.fit(X_train, y_train)

Fitting 5 folds for each of 1920 candidates, totalling 9600 fits
[CV] END algorithm=auto, metric=euclidean, n_neighbors=3, weights=uniform; total time=  34.9s
[CV] END algorithm=auto, metric=euclidean, n_neighbors=9, weights=distance; total time=  30.4s
[CV] END algorithm=auto, metric=manhattan, n_neighbors=3, weights=distance; total time= 3.8min
[CV] END algorithm=auto, metric=manhattan, n_neighbors=11, weights=uniform; total time= 3.9min
[CV] END algorithm=auto, metric=chebyshev, n_neighbors=3, weights=distance; total time= 3.9min
[CV] END algorithm=auto, metric=chebyshev, n_neighbors=11, weights=uniform; total time= 3.8min
[CV] END algorithm=auto, metric=minkowski, n_neighbors=3, weights=uniform; total time=  29.8s
[CV] END algorithm=auto, metric=minkowski, n_neighbors=5, weights=distance; total time=  29.6s
[CV] END algorithm=auto, metric=minkowski, n_neighbors=9, weights=distance; total time=  29.3s
[CV] END algorithm=auto, metric=minkowski, n_neighbors=13, weights=distance; total

/home/vtpldedpy/jupyter-test/venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
2400 fits failed out of a total of 9600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1679 fits failed with the following error:
Traceback (most recent call last):
  File "/home/vtpldedpy/jupyter-test/venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/vtpldedpy/jupyter-test/venv/lib/python3.11/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/vtpldedpy/jupyter-test/venv/lib/python3.11/site-packages/sklearn/base.py", line 666, in _validate_params
 

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20, 30, 40, 50],
                         'max_features': [None, 'auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 5, 10],
                         'min_samples_split': [2, 5, 10, 15, 20],
                         'splitter': ['best', 'random']},
             verbose=2)

In [34]:
best_params_dt = grid_search_dt.best_params_
best_score_dt = grid_search_dt.best_score_

print(f"Best parameters found: {best_params_dt}")
print(f"Best cross-validation score: {best_score_dt}")

Best parameters found: {'criterion': 'entropy', 'max_depth': 30, 'max_features': None, 'min_samples_leaf': 5, 'min_samples_split': 10, 'splitter': 'best'}
Best cross-validation score: 0.8569206349206349


In [37]:
best_model_dt = grid_search_dt.best_estimator_
best_model_dt.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=30, min_samples_leaf=5,
                       min_samples_split=10)

In [39]:
test_accuracy_dt = best_model_dt.score(X_test, y_test)
print(f"Test accuracy: {test_accuracy_dt:.4f}")

Test accuracy: 0.8617
[CV] END criterion=entropy, max_depth=40, max_features=log2, min_samples_leaf=5, min_samples_split=2, splitter=best; total time=   0.5s
[CV] END criterion=entropy, max_depth=40, max_features=log2, min_samples_leaf=5, min_samples_split=15, splitter=best; total time=   0.5s
[CV] END criterion=entropy, max_depth=40, max_features=log2, min_samples_leaf=10, min_samples_split=5, splitter=best; total time=   0.5s
[CV] END criterion=entropy, max_depth=40, max_features=log2, min_samples_leaf=10, min_samples_split=15, splitter=random; total time=   0.5s
[CV] END criterion=entropy, max_depth=50, max_features=None, min_samples_leaf=1, min_samples_split=5, splitter=random; total time=   3.1s
[CV] END criterion=entropy, max_depth=50, max_features=None, min_samples_leaf=1, min_samples_split=15, splitter=best; total time=   8.9s
[CV] END criterion=entropy, max_depth=50, max_features=None, min_samples_leaf=2, min_samples_split=10, splitter=random; total time=   2.6s
[CV] END crite